In [1]:
import pandas as pd
import numpy as np
from supporting_functions import *

/Volumes/TONY/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
neiss2014 = getTable()
dicts = getDicts()
dc_dict = dicts['diag']
disp_dict = dicts['disposition']

# Q1

In [3]:
# Q1
highest_freq_bp = getHiLoFreqPerCol('body_part','highest',3)
lowest_freq_bp = getHiLoFreqPerCol('body_part','lowest',3)
print 'Top three body parts most frequently represented in this dataset are: ' + ",".join(highest_freq_bp)
print 'Top three body parts least frequently represented in this dataset are: ' + ",".join(lowest_freq_bp)

Top three body parts most frequently represented in this dataset are: Finger,Face,Head
Top three body parts least frequently represented in this dataset are: 25-50% of body,Pubic region,Not Recorded


 # Q2

In [4]:
#Q2
# Counts for skateboards
print '2.a'
skb_indexes = getKeywordIndexes("skateboard")
print 'There are ' + str(len(skb_indexes)) + ' injuries in this dataset that involve a skateboard'+'\n'
skb_sex = getColumnData('sex',skb_indexes)
result = calcItemsFreq(skb_sex)

print '2.b'
print 'Of those injuries, the percentage of male and female are:'
for sex, percentage in result.items():
    print sex +':'+ str(percentage)
    
print '\n2.c'
skb_average_age = np.average(getColumnData('age',skb_indexes))
print 'Average age of someone injured in an incident involving a skateboard is ' + str(skb_average_age)

2.a
There are 466 injuries in this dataset that involve a skateboard

2.b
Of those injuries, the percentage of male and female are:
Male:0.824034334764
Female:0.175965665236

2.c
Average age of someone injured in an incident involving a skateboard is 17.991416309


# See the content of the dictionary

In [5]:
# See dictionary mapping between disposition and code, we knonw that 6 is corresponding to 
# 'Left without being seen/Left against medical advice'
disp_dict = dicts['disposition']
print disp_dict

{1: 'Treated and released, or examined and released\n without treatment', 2: 'Treated and transferred to another hospital', 4: 'Treated and admitted for hospitalization (within same facility)', 5: 'Held for observation (includes admitted for observation)', 6: 'Left without being seen/Left against medical advice', 8: 'Fatality, including DOA, died in the ED', 9: 'Not Recorded'}


# Q3

In [6]:
#Q3
print '3.a'
p3a = getAwithHiLoB('diag','disposition',itemInB=4,topN=1,freqOrCount='freq',HighestOrLowest='highest')
print ','.join(p3a) + ' had the highest hospitalization rate'

print '3.b'
p3a = getAwithHiLoB('diag','disposition',itemInB=6,topN=1,freqOrCount='count',HighestOrLowest='highest')
print ','.join(p3a) + ' is most often concluded with the individual leaving without being seen'

3.a
Submersion (including Drowning) had the highest hospitalization rate
3.b
Other/Not Stated is most often concluded with the individual leaving without being seen


## Q3.c
### The caveat should be, should we take the overall number of each feature into account, when we met words in 'rate' and 'most often'. The answer is, when we see 'rate', that suggests we should use percentage to rank our data, while 'most often' always refers to the time of occurances for one specific data.

In [7]:
age_data = getColumnData('age')
diagnosis_data = getColumnData('diag')

indexes_trueages = np.where(age_data<150)[0].tolist()
diagnosis_label = np.array([dc_dict[i] for i in diagnosis_data])

true_ages = age_data[indexes_trueages]
diag_trueages = diagnosis_label[indexes_trueages]

In [8]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

plotly.tools.set_credentials_file(username='tonyabracadabra', api_key='6gs9i5iec7')

trace = go.Scatter(
    x = age_data[indexes_trueages],
    y = diagnosis_label[indexes_trueages],
    mode = 'markers'
)

data = [trace]

py.iplot(data, filename='Age-diagnosis')

/Volumes/TONY/anaconda/lib/python2.7/site-packages/plotly/plotly/plotly.py:236: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



KeyboardInterrupt: 

In [9]:
trace = go.Histogram(
    x = age_data,
)

data = [trace]
py.iplot(data, filename='Distribution of ages')

# It therefore suggests that some data points assoicated with age might not be used since they are above 200 (not possible for ages), we should therefore eliminate those data points

In [10]:
bins = []
for i in xrange(10):
    bins.append(np.where((true_ages<i*10+10) & (true_ages>i*10)))

counts_diag_for_ages = [(diag,true_ages[np.where(diag_trueages==diag)[0].tolist()]) for diag in np.unique(diag_trueages)]
plot_dict_for_diagnosis = [(str(i)+'-'+str(i+10),diag_trueages[bins[i]]) for i in xrange(10)]
average_diag_for_ages = [(item[0],np.average(item[1])) for item in counts_diag_for_ages]

In [11]:
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

data_hist_ages = []
for item in counts_diag_for_ages:  
    trace = go.Histogram(
        x=item[1],
        opacity=0.75,
        name=item[0]
    )
    data_hist_ages.append(trace)

layout = go.Layout(
    title='Distribution Analysis',
    xaxis=dict(
        title='Age distribution for different diagnosis'
    ),
    yaxis=dict(
        title='Count in bin of 20'
    ),
    barmode='overlay',
    bargap=0.5,
    bargroupgap=20
)

fig = go.Figure(data=data_hist_ages, layout=layout)
py.iplot(fig)

In [12]:
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

data_hist_ages = []
x = [i[0] for i in plot_dict_for_diagnosis]
for item in plot_dict_for_diagnosis:  
    trace = go.Histogram(
        x=item[1],
        opacity=0.75,
        name=item[0]
    )
    data_hist_ages.append(trace)

layout = go.Layout(
    title='Distribution Analysis for Diagnosis in different ages',
    xaxis=dict(
        title='Diagnosis distribution for ages in range of 10'
    ),
    yaxis=dict(
        title='Count for each diagnosis'
    ),
    barmode='overlay',
    bargap=0.5,
    bargroupgap=20
)

fig = go.Figure(data=data_hist_ages, layout=layout)
py.iplot(fig)

ConnectionError: HTTPSConnectionPool(host='plot.ly', port=443): Max retries exceeded with url: /clientresp (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x11de3b9d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

# Q5

In [ ]:
X, y = getCleanData('disposition')

In [ ]:
weights = runDFS(X,y)

# I used a feature selection framework implemented by myself, to select the most important feature that could contribute to the model, at here, I choose disposition as labels and other columns as features to be learned in the model

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

plotly.tools.set_credentials_file(username='tonyabracadabra', api_key='6gs9i5iec7')

data = [
    go.Heatmap(
        z = np.array(np.abs(weights))
    )
]

py.iplot(data, filename='Weights from DFS stacked')

## That suggest body_part almost dominantly determine the disposition type, we can apply this to any types of labels and select the best feature through this way, I have used this method to solve many difficult problems w.r.t feature engineering

In [ ]:
words_list = [set(re.split('\W+',i)) for i in getColumnData('narrative')]
union_words = reduce(lambda x,y:x.union(y), words_list)
union_words.remove('')

word_hot_vector = np.array(word_hot_vector)
sparse_index1 = np.array(sparse_index1)
sparse_index2 = np.array(sparse_index2)

# word_hot_vector = [[1 if word in words else 0 for word in union_words] for words in words_list]

sparse_index1, sparse_index2 = [], []
for i, words in enumerate(words_list):
    if i % 1000 == 0:
        print i
    for j, word in enumerate(union_words):
        if word in words:
            sparse_index1.append(i)
            sparse_index2.append(j)

In [ ]:
np.save("narratives_hot_vector.npy",word_hot_vector)

In [ ]:
np.array(weights)[:,[1,2,3]].shape